In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down, coupled_flux_qubit_non_linear_approx_force_break_down, coupled_flux_qubit_non_linear_approx_pot_only_d_beta, coupled_flux_qubit_non_linear_approx_force_only_d_beta
import edward_tools.coupled_fq_potential as coupled_fq_potential_package

import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
# from PARAMETER_INPUT import *
# from PARAMETER_INPUT import _lambda, _theta, _eta
from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time, getFidelityTimeGraph

import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, datetime
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep



coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

In [ ]:

C_factor = 100
L_factor = 0.35
thermal_factor = 50

R_factor = 1
I_m_factor = 1
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

R_1, R_2 = 371 * R_factor, 371 * R_factor                                # ohm 

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
# time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
U0_1_k_BT_ratio = U0_1 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1 # these are kappa' in the equation

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;



_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4]) *thermal_factor
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4]) *thermal_factor

In [ ]:
d_beta_1 * U0_1_k_BT_ratio

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 10000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "test the effect of I_m asymmetry"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1


In [ ]:
m_2 = m_1 
m_3 = 1/4 * m_1
m_4 = 1/4 * m_2

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_only_d_beta, coupled_flux_qubit_non_linear_approx_force_only_d_beta, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)



In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_x_off = 0
phi_2_x_off = 0
phi_1_dcx_off = 0
phi_2_dcx_off = 0
M_12_off = 0

# bookmark
# phi_1_dcx_on = 2.8
# phi_2_dcx_on = 2.8
# M_12_on = -0.6

# phi_1_x_on = -0.008
# phi_2_x_on = -0.008
phi_1_x_on = -0.00
phi_2_x_on = -0.00
phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
zeroDissipation = False
saveAllStates = True

# params['circuit_parameter'] = {
#     "C_factor":C_factor, "L_factor": L_factor, "I_m_1_factor": I_m_factor, "T": T, 
#     "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
#     "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
#     "phi_1_dcx_on": phi_1_dcx_on, "phi_2_dcx_on": phi_2_dcx_on, "M_12_on": M_12_on
# }
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 1

# bookmark
# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
TR_initial_condition[0]

rest = lambda t: {"duration": t, "name":"rest"}
duration_t = 200
protocol_list = [
    # forward
    {"duration":200, "name": "test"},
    # {"duration":50, "phi_2_x": phi_1_x_on * 0.5,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":200,"phi_2_x": phi_1_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":60, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
    # {"duration":60, "phi_1_x": phi_1_x_on, "phi_2_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
    # {"duration":60, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
    # {"duration":200, "phi_1_x": phi_1_x_on * 0.5, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
    # {"duration":50,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
]

# protocol_list = [
#     # forward
#     {"duration":200, "phi_2_x": phi_1_x_on * 0.5,  "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
#     {"duration":200,"phi_2_x": phi_1_x_on, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
#     {"duration":200, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
    
#     {"duration":200, "phi_1_x": phi_1_x_on, "phi_2_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
#     {"duration":200, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
#     {"duration":200, "phi_1_x": phi_1_x_on * 0.5, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
#     {"duration":200,  "phi_1_x": phi_1_x_off, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
# ]



In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, measure_all_states=True)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(extra_constraint=extra_constraint_00_and_11_only)
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state
protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)

In [ ]:
def graphForCriticalValueOfPhi_xdc(parameterArray):
    XYU_array = []
    
    final_t = 40
    for p_array in parameterArray:   
        print(p_array)
        protocol_list = [
            {"duration":final_t, "phi_1_dcx": p_array[0], "phi_2_dcx": p_array[1],  "M_12": p_array[2],  "name":"(1) mix in y direction"},
        ]
        computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, protocol_list)
        storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)
        cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, storage_protocol= storage_protocol, computation_protocol= comp_protocol)
        cfqr.initialize_sim(); 
        cfqr.set_sim_attributes()
        init_state_saved = cfqr.init_state
        
        
        fig, ax = plt.subplots(1, 2, figsize = [12, 6])
        X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = final_t, cutlineDirection = "v")
        plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96436202,\
                                    contour_plt=ax[0], cutline_plt=ax[1], contours = 20,
                                    showGraph=True)

        XYU_array.insert(0, plt_line_data_at_t)
        
    return XYU_array

In [ ]:
# XYU_array = graphForCriticalValueOfPhi_xdc([(0, 2.6,0), (0, 2.8, 0), (0, 3.0, 0)])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = [14, 6])

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 50, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96436202,\
                            contour_plt=ax[0], cutline_plt=ax[1], contours = 20,
                            showGraph=True)

# contours, time = item["contour_plot"]["contours"], item["contour_plot"]["time"]
# _plotAxis = item["cutline_plot"]["plotAxis"]
# _targetU = item["cutline_plot"]["targetU"]
# ax[0].set_box_aspect(1)
# ax[1].set_box_aspect(1)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize = [14, 6])
# for index, item in enumerate(XYU_array):

#     X, Y, U = item["contour_plot"]["X"], item["contour_plot"]["Y"], item["contour_plot"]["U"]
#     contours, time = item["contour_plot"]["contours"], item["contour_plot"]["time"]
#     _plotAxis = item["cutline_plot"]["plotAxis"]
#     _targetU = item["cutline_plot"]["targetU"]
#     ax[0].set_box_aspect(1)
#     ax[1].set_box_aspect(1)

#     ax[0].set_xlabel(r"$\varphi_{1}$", fontsize = 20)
#     ax[0].set_ylabel(r"$\varphi_{2}$", fontsize = 20)
#     ax[0].xaxis.set_tick_params(labelsize=15)
#     ax[0].yaxis.set_tick_params(labelsize=15)

    
#     ax[1].set_ylabel("potential energy", fontsize = 18)
#     ax[1].set_xlabel(r"$\varphi_{2}$", fontsize = 20)
#     ax[1].xaxis.set_tick_params(labelsize=15)
#     ax[1].yaxis.set_tick_params(labelsize=15)

    
#     ax[0].contourf(X, Y, U, contours)
#     ax[0].vlines(x = -2.6, ymin = -5, ymax = 5, colors= "red")
#     ax[1].plot(_plotAxis, _targetU, label = r"$\varphi_{2dcx}$ = " + f"{phi_1_dcx_on_array[index]}")
#     # ax[1].legend(fontsize= 15)
# #     ax[1].set_title(, fontsize = 15)
# # "contour_plot": {"X": X, "Y": Y, "U": U, "contours": contours, "time": time},
# #         "cutline_plot": {"plotAxis": plotAxis, "targetU": targetU, "time": time, "cutlineDirection": cutlineDirection, "cutlineValue": cutlineValue}

# actual simulations

In [ ]:
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
mapping_state_1_to_state_2_dict = {"00": ["00", "10"], "01": ["00", "10"], "10": ["01", "11"], "11": ["01", "11"]}

In [ ]:
# step 5: perform simulations

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::], measure_all_states=saveAllStates)

# fidelity analysis

In [ ]:
import edward_tools.couple_flux_qubit_metrics as couple_flux_qubit_metrics

In [ ]:
print(f"phi_1_x_on: {phi_1_x_on}, phi_2_x_on: {phi_2_x_on}")

In [ ]:
simResult["fidelity"]


In [ ]:
getFidelityTimeGraph(simResult['cfqr'], params, mapping_state_1_to_state_2_dict)

In [ ]:
step_indices = np.array(simResult['cfqr'].sim.output.all_state["step_indices"])
time_step_array = step_indices * params['dt'] 

In [ ]:
step_indices_2 = step_indices[step_indices>0][::100]
time_step_array_2 = step_indices_2 * params['dt'] 
# time_step_array_2 = time_step_array[time_step_array > 500][::100]
metastable_state_array = ["00", "01", "10", "11"]
fidelity_array = []

for step in step_indices_2:
    _fidelity_at_time_t = couple_flux_qubit_metrics.fidelityEvaluation(simResult['cfqr'].init_state[:, (0, 1), :], 
                   simResult['cfqr'].sim.output.all_state["states"][:, step, (0, 1), :] , mapping_state_1_to_state_2_dict)
    fidelity_array.append([d['final_percentage'] for d in _fidelity_at_time_t])
fidelity_array = np.array(fidelity_array, dtype = float)

fig, ax = plt.subplots(2, 2, figsize = [10, 8])
ax_flatten = ax.flatten()

for i, _ax in enumerate(ax_flatten):
    _ax.plot(time_step_array_2, fidelity_array[:, i, 0], label = "00")
    _ax.plot(time_step_array_2, fidelity_array[:, i, 1], label = "01")
    _ax.plot(time_step_array_2, fidelity_array[:, i, 2], label = "10")
    _ax.plot(time_step_array_2, fidelity_array[:, i, 3], label = "11")
    _ax.set_title(f"initial_at_{metastable_state_array[i]}")
    _ax.legend()
plt.show()

In [ ]:
def showFidelity(target):

    final_percentage_matrix = np.array([d["final_percentage"] for d in target]).T
    text_box_array = ["\n".join(row) for row in final_percentage_matrix]
    # params_list = list(target["params"])[0]
    fig, ax = plt.subplots(1, 1, figsize = [4, 4])
    # plt.figure(figsize=[4, 4])
    # plt.title(f"N = {params_list['N']}, dt = {params_list['dt']}")
    ax.set_xlim(-5, 5);plt.ylim(-5, 5);
    ax.text(-1.25, -1, "00"); ax.text(-1.25, 4, "01")
    ax.text(4, -1, "10"); ax.text(4, 4, "11")
    ax.text(-3.25, -3.5, text_box_array[0]); ax.text(-3.25, 1.5, text_box_array[1])
    ax.text(1.75, -3.5, text_box_array[2]); ax.text(1.75, 1.5, text_box_array[3])
    ax.vlines(0, ymin = -5, ymax = 5); ax.hlines(0, xmin = -5, xmax = 5)

import edward_tools.couple_flux_qubit_metrics as couple_flux_qubit_metrics
couple_flux_qubit_metrics.fidelityEvaluation
fidelity_at_t_array = []
initial_state = simResult['cfqr'].init_state[:, (0, 1), :]
for t_index in protocol_time_index_array[-1:]:
    state_at_t = simResult['cfqr'].sim.output.all_state["states"][:, t_index, (0, 1), :]
    fidelity_at_t = couple_flux_qubit_metrics.fidelityEvaluation(initial_state, state_at_t, mapping_state_1_to_state_2_dict)
    showFidelity(fidelity_at_t)
    fidelity_at_t_array.append(fidelity_at_t)

# Work don analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')

In [ ]:
step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.01)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

coarse_step_array = step_array[::skip_step]
coarse_step_array = np.append(coarse_step_array, step_array[-1])
coarse_work_mean_array = work_mean[::skip_step]
coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', label = 'axvline - full height')
    # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
    plt.annotate(f"{i+1}", 
            xy=(t, 4.5),  
            # xytext=(quantile_75, 2.05*1e9),
            color='red', 
            ha='center', 
            size=16)

first_half = work_mean[0: len(work_mean)//2]
second_half = work_mean[len(work_mean)//2:][::-1]
abs_error = np.abs(first_half - second_half)
# plt.plot(step_array[0:len(step_array)//2], abs_error)
print(max(abs_error), min(abs_error))
print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:

target_time_index = np.array(protocol_time_array)/params['dt']
target_time_index = target_time_index.astype("int") - 1

work_done_array = [work_mean[i] for i in target_time_index[1:]]
work_done_array.insert(0, 0)
work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

Measure_Erasue_steps = [0, 1, 2, 3]
Control_Steps = [4, 5, 6]



print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
print(f"work_done = {work_done_of_each_step}" )
# print("measurement + reset = ", work_done_of_each_step[0] + work_done_of_each_step[1] + work_done_of_each_step[2] )
# print("control = ", work_done_of_each_step[3] + work_done_of_each_step[4] + work_done_of_each_step[5] + work_done_of_each_step[6])
# print(sum(work_done_of_each_step))


print("measurement + reset = ", np.sum([work_done_of_each_step[i] for i in Measure_Erasue_steps]))
print("control = ", np.sum([work_done_of_each_step[i] for i in Control_Steps]))
print(sum(work_done_of_each_step))

In [ ]:
protocol_time_array

In [ ]:
def get_work_done_from_time_array(params, work_statistic_array, protocol_time_array):
    target_time_index = np.array(protocol_time_array)/params['dt']
    target_time_index = target_time_index.astype("int") - 1
    
    work_done_array = [work_statistic_array[i] for i in target_time_index[1:]]
    work_done_array.insert(0, 0)
    work_done_array = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]
    print(target_time_index, work_done_array)

# animations

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 2000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery")

# minimum value of potential energy

In [ ]:
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
import importlib
importlib.reload(minimum_value_of_potential)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[9, 4])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[0], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points_location[0][0],\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value,
                            showGraph=True)
energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
print("min U: ", np.min(U))

## mix in Y

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=[9, 4])
# ax_flatten = ax.flatten()
# X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[1], cutlineDirection = "v")
# plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points[1][0],\
#                             contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value, 
#                             showGraph=True)
# energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
# print("min U: ", np.min(U))

fig, ax = plt.subplots(1, 2, figsize=[9, 4])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[1], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points_location[1][0],\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value, 
                            showGraph=True)
energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
print("min U: ", np.min(U))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[9, 4])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[2], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points_location[2][0],\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value, 
                            showGraph=True)
energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
print("min U: ", np.min(U))

## conditional_tilt_in_Y

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=[9, 4])
# ax_flatten = ax.flatten()
# X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[2], cutlineDirection = "v")
# plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points[2][0],\
#                             contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value,  
#                             showGraph=True)
# energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
# print("min U: ", np.min(U))

fig, ax = plt.subplots(1, 2, figsize=[9, 4])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = protocol_time_array[3], cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = minimum_points_location[3][0],\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = contour_value,  
                            showGraph=True)
energy_profile_array.append([plt_line_data_at_t['cutline_plot']['plotAxis'], plt_line_data_at_t['cutline_plot']['targetU']])
print("min U: ", np.min(U))

# KE analysis

In [ ]:
def get_all_state(self):
    return self.sim.output.all_state['states']

def get_time_step_indices(self):
    time_indices_array = np.array(self.sim.output.all_state['step_indices'])
    time_array = time_indices_array * cfqr.sim.dt
    return time_indices_array, time_array

In [ ]:
all_state = get_all_state(simResult['cfqr'])
time_indice_array, time_array = get_time_step_indices(simResult['cfqr'])


In [ ]:
cfqr.sim.nsteps

In [ ]:
cfqr.sim.dt


In [ ]:
KE_time_array

In [ ]:
KE_time_array = np.empty([cfqr.sim.ntrials, cfqr.sim.nsteps + 1])

for _i, _t in zip(time_step_indices, time_array):
    KE_time_array[:, _i] = simResult['cfqr'].system.get_kinetic_energy(all_state[:,_i, ...])
    

In [ ]:
all_state[:, :, 0, 1]**2 / 2 + all_state[:, :, 1, 1]**2 / 2  + all_state[:, :, 2, 1]**2 / 2 + all_state[:, :, 3, 1]**2 / 2 

In [ ]:
simResult['cfqr'].sim.output.all_state

# Trident_analysis

In [ ]:
def work_categories(work_dist, all_states, bins = [0.2, 1.5]):
    """
    This function will separate the particles by categories and from that find out the thing that distinguish them from each other.
    
    work_dist_left = the distribution on the left of the trident distribution
    """
    
    work_dist_left = all_states[work_dist < bins[0]]
    work_dist_middle = all_states[np.logical_and(work_dist >= bins[0], work_dist <= bins[1])]
    work_dist_right = all_states[work_dist > bins[1]]
    
    return work_dist_left, work_dist_middle, work_dist_right
                                  
    

In [ ]:
work_dist_left, work_dist_middle, work_dist_right = work_categories(simResult["work_distribution"], simResult['cfqr'].sim.output.all_state['states'])

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 5000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = work_dist_middle
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
all_state = work_dist_left
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
all_state = work_dist_right
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = [12, 4])

for d in work_dist_left[::100]:
    ax[0].plot(d[:, 2, 0], "b")
    
for d in work_dist_right[::100]:
    ax[1].plot(d[:, 2, 0], "g")
    
for d in work_dist_middle[::100]:
    ax[0].plot(d[:, 2, 0], "r", alpha=0.1)
    ax[1].plot(d[:, 2, 0], "r", alpha=0.1)
    
    

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = [12, 4])

for d in work_dist_left[::100]:
    ax[0].plot(d[50000:, 3, 0], "b")
    
for d in work_dist_right[::100]:
    ax[1].plot(d[50000:, 3, 0], "g")
    
for d in work_dist_middle[::100]:
    ax[0].plot(d[50000:, 3, 0], "r", alpha=0.2)
    ax[1].plot(d[50000:, 3, 0], "r", alpha=0.2)
    
    

In [ ]:
plt.plot(np.mean(work_dist_left[:, :, 2, 0], axis = 0))
plt.plot(np.mean(work_dist_middle[:, :, 2, 0], axis = 0))
plt.plot(np.mean(work_dist_right[:, :, 2, 0], axis = 0))


In [ ]:
protocol_time_array

In [ ]:
for d in work_dist_middle[::100]:
    plt.plot(d[:, 3, 0])
    
    

In [ ]:

work_dist_right_btw_CTX_and_MX = work_dist_right[:, 100000-1:120000-1, ...]
work_dist_middle_btw_CTX_and_MX = work_dist_middle[:, 100000-1:120000-1, ...]
work_dist_left_btw_CTX_and_MX = work_dist_left[:, 100000-1:120000-1, ...]

In [ ]:
fig, ax = plt.subplots(4, 3, figsize = [12, 12])
ax_flatten = ax.flatten()

for i in range(0, 4):
    ax_flatten[3*i].hist(work_dist_left_btw_CTX_and_MX[:, 0, i, 0], bins = 20)
    ax_flatten[3*i+1].hist(work_dist_middle_btw_CTX_and_MX[:, 0, i, 0], bins = 20)
    ax_flatten[3*i + 2].hist(work_dist_right_btw_CTX_and_MX[:, 0, i, 0], bins = 20)


# 

In [ ]:
protocol_time_all_index_array

In [ ]:
protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_all_time_index_array = cfqr.createProtocolTimeArray(protocol_list, params)

target_filter = np.logical_and(protocol_all_time_array >= 1000, protocol_all_time_array <= 1200)
target_time_array = protocol_all_time_array[target_filter]
target_time_index_array = protocol_all_time_index_array[target_filter]
work_dist_left_in_t_range = work_dist_left[:, target_filter, ...]
work_dist_middle_in_t_range = work_dist_middle[:, target_filter, ...]
work_dist_right_in_t_range = work_dist_right[:, target_filter, ...]

In [ ]:
from matplotlib.animation import FuncAnimation


def plot_work_dist_with_time(cfqr, work_dist, pot_function, target_time_array, target_time_index_array, skipNumber = 250):
    manual_domain=[np.array([-3, -3]), np.array([3, 3])]
    modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), (manual_domain[1][1], manual_domain[1][0])]

    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    resolution = 50
    contours = 20
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    _skipped_target_time_array = target_time_array[::skipNumber]
    _skipped_target_time_index_array = target_time_index_array[::skipNumber]
    
    x_coordinate_of_dist_array = work_dist[:, :, 0, 0]
    U_of_dist_array = getEnergyTimeArrayOfDist(cfqr, work_dist, target_time_array, target_time_index_array, skipNumber)
    
    ax.scatter(x_coordinate_of_dist_array[0], U_of_dist_array[0])
    
    
    def animate(i):
        ax.clear()
        # Get the point from the points list at index i
        _t = _skipped_target_time_array[i]
        _params_at_t = cfqr.protocol.get_params(_t)
        beta_1 = _params_at_t[4]
        beta_2 = _params_at_t[5]
        d_beta_1 = _params_at_t[6]
        d_beta_2 = _params_at_t[7]
        _phi_1x = _params_at_t[8]
        _phi_2x = _params_at_t[9]
        _phi_1dcx = _params_at_t[10]
        _phi_2dcx = _params_at_t[11]
        _M_12 = _params_at_t[12]
        _xi = 1 / (1 - _M_12**2)
        slice_values = [0, 0, _phi_1dcx, _phi_2dcx]
        
        U, X_mesh = cfqr.system.lattice_with_self_defined_potential(_t, pot_function, resolution, axes=(0, 1),\
                                    manual_domain=modified_manual_domain, slice_values = slice_values)
        x_array, y_array = X_mesh
        
        selected_U = U[np.all(np.logical_and(y_array > 0.5, y_array < 3), axis = 1)]
        selected_x = x_array[np.all(np.logical_and(y_array > 0.5, y_array < 3), axis = 1)]

        ax.set_title(f"t = {_t}")
        for _x, _U in zip(selected_x, selected_U):
            ax.plot(_x, _U)
        ax.scatter(x_coordinate_of_dist_array[i], U_of_dist_array[i])
    
    ani = FuncAnimation(fig, animate, frames=len(_skipped_target_time_index_array),
                        interval=100, repeat=False)
    plt.close()
    return ani



In [ ]:
target_time_index_array

In [ ]:
ani = plot_work_dist_with_time(cfqr, work_dist_left, coupled_flux_qubit_non_linear_approx_pot,
                        target_time_array, target_time_index_array)

from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)



In [ ]:
def getEnergyTimeArrayOfDist(cfqr, work_dist, target_time_array, target_time_index_array, skipNumber = 1):
    U_of_dist_time_array = np.empty([work_dist.shape[0], len(target_time_index_array[::skipNumber])])
    
    for _j, _t_index in enumerate(target_time_index_array[::skipNumber]):
        _t = target_time_array[_t_index]
        U_of_dist_time_array[:, _j] =  cfqr.system.get_potential(work_dist[:, _t_index, ...], _t)
    return U_of_dist_time_array

In [ ]:
getEnergyTimeArrayOfDist(simResult['cfqr'], work_dist_left, protocol_all_time_array, protocol_all_time_index_array, skipNumber=10)

In [ ]:
cfqr.system.get_potential(work_dist[0, :, ...], _t)

In [ ]:
target_time_index_array[::10]

In [ ]:
cfqr.createProtocolTimeArray(protocol_list, params)

In [ ]:
work_dist_right.shape

In [ ]:
plt.plot(time_array[ 43000: 63000], work_dist_left[2, 43000: 63000, 0, 1])

In [ ]:
plt.plot(time_array[ 43000: 63000], work_dist_left[2, 43000: 63000, 0, 1])

In [ ]:
KE_0 = cfqr.system.get_kinetic_energy(work_dist_right)


In [ ]:
for x in KE_0[2:3]:
    plt.plot(x)

In [ ]:
plt.plot(KE_0[0])

In [ ]:
KE_p1 = 1/2 * work_dist_right[0,:, 0, 1]**2

In [ ]:
plt.plot(KE_p1)

In [ ]:
all_state = work_dist_right[0:1,...]
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=1000, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
import numpy as np
from scipy.stats import maxwell
import matplotlib.pyplot as plt


In [ ]:
r_1 = maxwell.rvs(size=1000)
r_2 = -maxwell.rvs(size=1000)

In [ ]:
plt.hist(np.array([r_1, r_2]).flatten(), bins = 60)

In [ ]:
plt.hist(r) 